In [7]:
import os
if os.path.basename(os.getcwd()) == "notebooks":
    os.chdir("..")
%cd .
%load_ext autoreload
%autoreload 2

/home/jh/code/til/til-23-cv


## YOLO Object Detection


In [4]:
from ultralytics import YOLO
from PIL import Image
from time import sleep
from pathlib import Path

In [3]:
model = YOLO("yolov5m6u.pt")

### See Model


In [ ]:
result = model.predict(
    "http://farm5.staticflickr.com/4032/5134536454_184138d8b1_z.jpg",
    half=True,
    device="cuda:0",
)

In [ ]:
def imshow(arr):
    display(Image.fromarray(arr[..., ::-1]))


imshow(result[0].plot())

### Preview Augmentations


In [ ]:
def end_early(_):
    raise KeyboardInterrupt

tempmodel = YOLO("yolov8n.pt")
tempmodel.add_callback("on_train_batch_end", end_early)
try:
    tempmodel.train(
        cfg="cfg/custom.yaml",
        name="temp/temp",
        deterministic=True,
        workers=0,
        batch=9,
        amp=False,
        verbose=False,
        device="cpu",
    )
except KeyboardInterrupt:
    pass
sleep(1)
for im in list(tempmodel.trainer.plots.keys())[1:]:
    display(Image.open(im))

### Train Model


In [ ]:
# TODO: See trained model perform on test split using eyeballs
model.train(
    cfg="cfg/custom.yaml",
    data="data/til23plushonly/dataset.yaml",
    name="custom/yolov5m6u",
    workers=8,
    batch=8,
)

### Validate Model

In [5]:
# NOTE: `runs/detect/custom/yolov5m6u/weights/best.pt` was renamed to `custom_yolov5m6u_best.pt`.
model_ckpt = "custom_yolov5m6u_best.pt"
im_root = "data/til23plush/images"
split = "test"

In [8]:
model = YOLO(model_ckpt)

In [ ]:
# NOTE: IT WILL APPEND TO EXISTING PREDICTIONS. MOVE CURRENT PREDICTIONS ELSEWHERE FIRST.
model.fuse()
results = model.predict(
    source=str(Path(im_root) / split),
    conf=0.7,
    iou=0.7,
    half=True,
    save_txt=True,
    save_conf=True,
    device="cuda:0",
    imgsz=1280,
    stream=True,
    name=f"predict/{split}",
)
for _ in results:
    pass